### LSTM

In [11]:
import pandas as pd
import numpy as np
data = pd.read_csv('BTC-Indicators.csv', parse_dates=['Date'], index_col=['Date'])
data

,Close,sma,ema20,ema50,ema200,rsi,macd,macd_signal,Close_future,Price_Up
Date,,,,,,,,,,
2015-04-04,253.697006,266.058734,255.661524,258.734938,307.338692,47.508324,-4.193133,-3.906538,260.597992,1
2015-04-05,260.597992,265.654567,256.131663,258.807999,306.873610,52.432292,-3.252122,-3.775655,255.492004,0
2015-04-06,255.492004,264.962268,256.070744,258.677960,306.362350,48.785829,-2.885117,-3.597547,253.179993,0
2015-04-07,253.179993,264.256468,255.795434,258.462354,305.833173,47.185618,-2.749132,-3.427864,245.022003,0
2015-04-08,245.022003,262.770301,254.769393,257.935281,305.228086,41.956153,-3.262042,-3.394700,243.675995,0
...,...,...,...,...,...,...,...,...,...,...
2024-12-12,100043.000000,95711.704687,96622.993102,88474.185367,71606.585559,60.775607,3382.945703,3915.653792,101459.257812,1
2024-12-13,101459.257812,96074.207812,97083.589741,88983.403894,71903.627074,62.898526,3378.370898,3808.197213,101372.968750,0
2024-12-14,101372.968750,96544.959115,97492.102028,89469.269183,72196.854355,62.675951,3329.403238,3712.438418,104298.695312,1


In [12]:
features = ['Close', 'sma', 'ema20', 'ema50', 'ema200', 'rsi', 'macd', 'macd_signal']
target = ['Price_Up']
# Separar X e y
X = data[features]
y = data[target].to_numpy()
y.shape, y

((3545, 1),
 array([[1],
        [0],
        [0],
        ...,
        [1],
        [1],
        [1]]))

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
X_scaled = scaler.fit_transform(X) # Ya lo convierte en un array de numpy 
X_scaled.shape, X_scaled

((3545, 8),
 array([[-0.99918348, -0.99926728, -0.9994773 , ..., -0.10909682,
         -0.16561299, -0.18373406],
        [-0.99905305, -0.99927559, -0.99946777, ...,  0.0076098 ,
         -0.16545748, -0.18370997],
        [-0.99914955, -0.99928982, -0.99946901, ..., -0.07881772,
         -0.16539683, -0.18367719],
        ...,
        [ 0.91198669,  0.98032724,  0.97162803, ...,  0.2504024 ,
          0.38528123,  0.50024848],
        [ 0.96728338,  0.9893965 ,  0.98476881, ...,  0.3516285 ,
          0.41124025,  0.49193136],
        [ 1.        ,  1.        ,  1.        , ...,  0.40486492,
          0.44783224,  0.49342827]]))

In [52]:
sequence_length = 30 # Ventana temporal de 30 dias 
X_sequences = []
y_sequences = []
for i in range(sequence_length, len(X_scaled)): # X_scaled tiene dimension de 1000, X_sequences tendra 1000 - 30(sequence_leght)
    X_sequences.append(X_scaled[i - sequence_length: i])
    y_sequences.append(y[i])
X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)
X_sequences.shape, y_sequences.shape


((3515, 30, 8), (3515, 1))

In [53]:
n = len(X_sequences)
n_train = int(n * 0.8)  # 80%
X_train = X_sequences[:n_train]
y_train = y_sequences[:n_train]
X_test = X_sequences[n_train:]
y_test = y_sequences[n_train:]

### Implementacion con Pytorch

In [54]:
# Convertimos los datos de Numpy a Tensores de Pytorch 
import torch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape, y_test_tensor.shape 

(torch.Size([2812, 30, 8]),
 torch.Size([703, 30, 8]),
 torch.Size([2812, 1]),
 torch.Size([703, 1]))

In [55]:
# Creamos Datasets y Dataloaders de Pytorch
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor) # Instanciamos la clase TendorDataSet 
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size, shuffle=True) # Instanciamos la clase DataLoader
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

print(len(train_loader))
for batch_X, batch_y in train_loader:
    print("Batch X shape:", batch_X.shape)
    print("Batch y shape:", batch_y.shape)
      # Solo el primer batch
    break


88
Batch X shape: torch.Size([32, 30, 8])
Batch y shape: torch.Size([32, 1])


In [56]:
import torch
import torch.nn as nn 

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print (device)

class MiLSTM (nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size ):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        # Definimos 1 capa LSTM y un Full Connected 
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) #Ej [8, 64, 2, (batch_size, seq_length, hidden_size)]
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Estados ocultos y celdas en 0 (num_layers, batch_size, hidden_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # Ej [2, 32, 64]
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device) # Ej [2, 32, 64]

        # Pasa la entrada por la capa LSTM 
        out, _ = self.lstm(x, (h0,c0)) #LLamo a la instancia de lstm (llamo a su metodo __call__ (no necesita nombre)) (batch_size, seq_length, hidden_size) [32, 30, 64]

        #Seleccionamos la ultima salida de la secuencia
        out = out[:,-1,:] # (batch_size , hidden_size) [32,64]

        #Pasa por la fc
        out = self.fc(out) # (batch_size, output_size) Ej [32,1]

        return out

# Parametros del modelo 
input_size = 8 
hidden_size = 64 
num_layers  = 2
output_size = 1 # Clasificacion 

model = MiLSTM(input_size, hidden_size, num_layers, output_size).to(device)
print(model) 

cuda
MiLSTM(
  (lstm): LSTM(8, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)


In [57]:
# Definir la función de pérdida y el optimizador
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento del modelo
num_epochs = 50 
model.train()

for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        batch_X = batch_X.to(device) #[32, 30, 8]
        batch_y = batch_y.to(device) #[32, 1]

        outputs = model(batch_X).squeeze() # [batch_size, 1] => [batch_size] 

        loss = criterion(outputs, batch_y.squeeze())

        loss.backward()

        optimizer.step()

        optimizer.zero_grad()
    if (epoch + 1 ) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')





Epoch [10/50], Loss: 0.7140
Epoch [20/50], Loss: 0.6772
Epoch [30/50], Loss: 0.6946
Epoch [40/50], Loss: 0.6973
Epoch [50/50], Loss: 0.6447


In [58]:
model.eval()  # Modo de evaluación
with torch.no_grad():
    correct = 0
    total = 0
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)

        outputs = model(batch_X).squeeze()   # Logits
        probs = torch.sigmoid(outputs)       # Probabilidades
        predicted = torch.round(probs)       # Predicciones binarias (0 o 1)

        total += batch_y.size(0)
        correct += (predicted == batch_y.squeeze()).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on test set: {accuracy:.2f}%')

Accuracy on test set: 50.07%
